In [21]:
import requests
import os
from bs4 import BeautifulSoup
import re

In [22]:
tagPageUrlArray = ["https://classics.autotrader.com/classic-cars-for-sale?seller_type=seller&distance=100",
                   "https://classiccars.com/listings/find?auction=false&dealer=false&private=true",
                   "http://www.rvt.com/New-and-Used-RVs-For-Sale-On-RVT.com/results?private_only=1",
                   "https://www.cars.com/for-sale/searchresults.action/?zc=60606&rd=99999&slrTypeId=28879&yrId=58487&yrId=56007&yrId=51683&yrId=47272&yrId=39723&yrId=34923&yrId=27381&yrId=20201&yrId=20145&yrId=20200&yrId=20144&yrId=20199&yrId=20143&yrId=20198&yrId=20142&yrId=20197&yrId=20141&yrId=20180&stkTypId=28881&sf1Nm=price&sf1Dir=DESC&sf2Nm=miles&sf2Dir=ASC&perPage=50&sortFeatures=buryUsedLowPrice&sortFeatures=buryNewLowPrice&sortFeatures=buryLowPriceOlderThanSix&sortFeatures=buryNoPrice&sortFeatures=buryUsedLowMileage&searchSource=GN_BREADCRUMB",
                   "http://www.ksl.com/auto/search/index?keyword=&yearFrom=&yearTo=&mileageFrom=&mileageTo=&priceFrom=25000&priceTo=1000000&zip=&miles=25&newUsed%5B%5D=All&sellerType%5B%5D=For+Sale+By+Owner&postedTime%5B%5D=&titleType%5B%5D=&body%5B%5D=&transmission%5B%5D=&cylinders%5B%5D=&liters%5B%5D=&fuel%5B%5D=&drive%5B%5D=&numberDoors%5B%5D=&exteriorCondition%5B%5D=&interiorCondition%5B%5D=&cx_navSource=hp_search",
                   "http://www.cars-on-line.com/search.html"]
max_page = 41

In [23]:
bigFile = open('ksl.txt', 'a')

In [24]:
for i in range(1, max_page):
    tagPageUrl = tagPageUrlArray[4]+"&page="+str(i)

    # Retrieve the page with tag results and set it up to be scraped
    tagPage = requests.get(url=tagPageUrl)
    tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

    links = tagPageSoup.find_all("div.photo-block > a")
    for link in links:
        url = "http://www.ksl.com" + link['href']
        page = requests.get(url=url)
        soup = BeautifulSoup(page.content, 'lxml')
        model = soup.select("#titleMain > h1 > div")
        if(len(model)):
            model = model[0].get_text().strip()
        else:
            model = "Unknown"
        price = soup.select("#titleMain > h3")
        if(len(price)):
            price = price[0].get_text().strip()
        else:
            price = "Unknown"
        description = soup.select('#widgetDescription > div.short.cXenseParse')
        if(len(description)):
            description = description[0].get_text().strip()
        else:
            description = ""
        if(len(description)):
            match = re.search(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}", description)
            if match:
                phone = match.group()
            else:
                phone = ""
            if len(phone):
                bigFile.write(model + '\t' + price + '\t' + phone + '\n')

In [47]:
bigFile.close()